### Crawler

In [35]:
class Crawler:
    
    def __init__(self):
        
        import time
        from urllib.request import Request, urlopen
        
        self.max_depth = 20
        self.page_hash = {}
        self.tracklist_hash = {}
        
    def find_str(self, s, char, start_index=0):

        index = 0
        s = s[start_index+1:]
        if char in s:
            c = char[0]
            for ch in s:
                if ch == c:
                    if s[index:index+len(char)] == char:
                        return start_index + 1 + index
                index += 1
        return -1   

    def request(self,url):

        req = Request(url,\
                      headers={'User-Agent': 'Mozilla/5.0'})
        html = str(urlopen(req).read())
        return html
    
    def parse(self):
        
        '''
        METHOD FOR PARSING HTML
        PROBABLY IMPLEMENT PARSER CLASS TO DECOUPLE PARSER AND CRAWLER
        MAKW ATTRIBUTE OF CLASS OR CALLBACK
        '''
        pass
    
    def crawl(self, url, depth):
        
        time.sleep(10)
        
        print('Depth:', depth)
        if depth == self.max_depth:
            return
        
        print(self.page_hash.get(url, None))
        # Check if already reached by search
        if self.page_hash.get(url, 0) == False:
            
            print(url)
            self.page_hash[url] = 1
            
            # Make http request
            html = self.request(url)
            index = 0
            # Iterate over links found in html
            while self.find_str(html, 'a href="', index) != -1:

                # Extract url
                index = self.find_str(html, 'a href="', index)
                url_chunk = html[index:].split('"')[1]
                
                # Make sure it is either a referenced tracklist or 1001 page
                if ('/tracklist/' in url_chunk) and ('http' not in url_chunk) and ('#tlp' not in url_chunk):
                    new_page = 'https://www.1001tracklists.com' + url_chunk
                    self.crawl(new_page, depth + 1)
                if ('www.1001tracklists.com' in url_chunk) and ('#tlp' not in url_chunk):
                    self.crawl(url_chunk, depth + 1)
                    
        return
        
    def start_crawl(self, startUrl):
        
        depth = 0
        self.crawl(startUrl, depth)
        
#crawler = Crawler()
#crawler.start_crawl('https://www.1001tracklists.com')

### Parser

- Should probably remove features from artist values
- and break up artists into multiple entries if there is like & or vs. or something

In [42]:
import ftfy
import urllib
from urllib.request import Request, urlopen

def find_str(s, char, start_index=0):
    '''
    Find substring char in string s. Found on internet, probably not efficient.
    
    '''
    index = 0
    s = s[start_index+1:]
    if char in s:
        c = char[0]
        for ch in s:
            if ch == c:
                if s[index:index+len(char)] == char:
                    return start_index + 1 + index
            index += 1
    return -1

def extract_value(html, key_value):
    
    content_string = html.strip(key_value).split('>')[0].strip('"')
    return content_string

def fix_decoding_errors(string):
    '''
    Fix UTF-8 decoding issues. Probably need to find more systematic/thorough approach to this.
    
    REPLACE THIS WITH ftfy.fix_text() -- python package which should be one stop shop for fixes
    '''
    # string = string.replace('&amp;','&')
    # string = string.replace('&#39;',"'")
    # string = string.replace('\\xc3\\xb6','o')
    # string = string.replace('\\xc3\\xab','e')
    
    return ftfy.fix_text(string)

def parse_track_and_artist(track_string):
    '''
    Extract the artist, track name, and remixer (if any) from the standard formatting used by 1001.
    
    '''
    # Check if Remix/Bootleg/Edit and parse accordingly
    if ('Remix' in track_string) or ('Bootleg' in track_string) or ('Edit' in track_string):
        
        # I think this section can/should be reworked
        # Should probably split around the dash first
        track_string = track_string.split('(')
        
        if len(track_string) == 3:
            track_string = [track_string[0] + '(' + track_string[1], track_string[2]]
        
        artist, track = [string.strip(' ') for string in track_string[0].split(' - ')]
        remixer = track_string[1].rstrip('Remix)').strip(' ')
        
    # If not remix, then should follow standard layout "Artist Name - Track Name"
    # This layout is expressed explicitly in html
    else:
        
        artist, track = [string.strip(' ') for string in track_string.split(' - ')]
        remixer = 'N/A'

    # Check for multiple artists -- Big Room sets tend to have hella mashups
    # Sometimes there is more structured formatting to exploit i.e. (Artist1 vs. Artist2 - Track1 vs. Track2)
    # Not worrying about that now b/c big room sux
    if 'vs.' in artist:
        artist = artist.replace('vs.','&')
    if '&' in artist:
        artist = [a.strip(' ') for a in artist.split('&')]
    
    # Remove features
    # We could make features a separate field but for now just removing
    if isinstance(artist, str):
        if ('feat.' in artist) or ('ft.' in artist):
            artist = artist.split('feat.')[0].strip(' ')
            artist = artist.split('ft.')[0].strip(' ')
    if isinstance(artist, list):
            artist = [a.split('feat.')[0].split('ft.')[0].strip(' ') for a in artist]
        
    if isinstance(artist, list):
        return (artist, track, remixer)
    else:
        return ([artist], track, remixer)


def tracklist_meta_data(html):
    '''
    Extract meta data about tracklist/set.
    
    '''
    meta_data = {}
    
    # Extract set description
    index = 0
    start_term = 'meta name="description" content="'
    index = find_str(html, start_term, index)
    description = html[index:].split('>')[0]
    description = description.lstrip(start_term).rstrip('"')
    meta_data['description'] = description
    
    # Set creation date - This should probably be the point in time we use for building prediction data
    index = 0
    start_term = 'meta name="dcterms.created" content="'
    index = find_str(html, start_term, index)
    created = html[index:].split('>')[0]
    created = created.lstrip(start_term).rstrip('"')
    meta_data['created'] = created
    
    # Set last modified data
    index = 0
    start_term = 'meta name="dcterms.modified" content="'
    index = find_str(html, start_term, index)
    modified = html[index:].split('>')[0]
    modified = modified.lstrip(start_term).rstrip('"')
    meta_data['modified'] = modified
    
    return meta_data
    
def tracklist_general_information(html):
    '''
    Extract general info about tracklist/set.
    
    '''
    info_doc = {}
    index = 0
    start_term = 'General Information'
    index = find_str(html, start_term, index)
    info_chunk = html[index:].split('Most Liked Tracklists')[0]
    
    # Genres -- can use these to build genre-specific graphs
    style_index = 0
    style_index = find_str(info_chunk, 'Tracklist Musicstyle', style_index)
    styles = info_chunk[style_index:].split('id="tl_music_styles">')[1].split('</td>')[0]
    styles = [style.strip(' ') for style in styles.split(',')]
    info_doc['styles'] = styles
    
    # If 1001 recognizes the dj who played the set they link their dj page
    # Its my understanding dj pages are independent of artist pages -- we'll need to map these
    index = 0
    start_term = 'a href="/dj'
    index = find_str(html, start_term, index)
    if index != -1:
        dj_url = html[index:].split('class')[0].split('"')[1]
        dj_url = 'https://www.1001tracklists.com' + dj_url
        info_doc['dj_url'] = dj_url

        dj_name = html[index:].split('</a>')[0].split('>')[1]
        info_doc['dj_name'] = dj_name
    else:
        info_doc['dj_url'] = 'N/A'
        info_doc['dj_name'] = 'N/A'
        
    return info_doc
    
def tracklist_track_data(html):
    '''
    Extract track related data from set
    '''
    track_docs = {}
    index = 0
    while find_str(html, 'tracknumber_value">', index) != -1:

        index = find_str(html, 'tracknumber_value">', index)
        #print(index)
        track_chunk = html[index:].split('<br>')[0]

        # Extract track number
        track_num = track_chunk[:22].split('<')[0].strip('tracknumber_value">')
        #print('Track Number:', track_num)

        # Extract track information
        chunk_index = 0
        chunk_index = find_str(track_chunk, 'meta itemprop="name" content=', chunk_index)
        extracted_value = track_chunk[chunk_index:].strip('meta itemprop="name" content=').split('>')[0].strip('"')
        clean_string = fix_decoding_errors(extracted_value)
        #print(clean_string)
        
        if len(clean_string) > 1:
            artist_list, track, remixer = parse_track_and_artist(clean_string)
        else:
            artist_list, track, remixer = None, None, None
            
        # Avoid ID's for now
        if artist_list is None:
            pass
        # If track info pull failed then pass
        elif (('ID' in artist_list) or ('ID' in track)): 
            pass
        else:
            
            # Tends to be multiple artists so artists parsed to list even if only one
            for artist in artist_list:
                
                #print('Artist:',artist)
                #print('Track:', track)
                #print('Remixer:', remixer)

                # Extract artist page
                artist_index = 0
                artist_index = find_str(track_chunk, 'title="open artist page"', artist_index)
                if artist_index != -1:
                    artist_url = track_chunk[artist_index:].split('class')[1].split('href="')[1].rstrip('" ')
                    #print('Aritst url:', artist_url)
                    artist_url = 'https://www.1001tracklists.com' + artist_url
                else:
                    artist_url = 'N/A'

                # Extract remixer page (if exists)
                if remixer != 'N/A':
                    remixer_index = find_str(track_chunk, 'title="open remixer artist page"', artist_index)
                    if remixer_index != -1:
                        remixer_url = track_chunk[remixer_index:].split('class')[1].split('href="')[1].rstrip('" ')
                        #print('Remixer url:', remixer_url)
                        remixer_url = 'https://www.1001tracklists.com' + remixer_url
                    else:
                        remixer_url = 'N/A'
                else:
                    remixer_url = 'N/A'

                # Extract track page
                track_index = 0
                track_index = find_str(track_chunk, 'title="open track page"', artist_index)
                if track_index != -1:
                    track_url = track_chunk[track_index:].split('class')[1].split('href="')[1].split('"')[0]
                    #print('track url:', track_url)
                    track_url = 'https://www.1001tracklists.com' + track_url
                else:
                    track_url = 'N/A'

                track_doc = {\
                            'track_num': int(track_num),
                            'artist': artist.strip(' '),
                            'artist_url': artist_url.strip(' '),
                            'name': track.strip(' '),
                            'track_url': track_url.strip(' '),
                            'remixer': remixer.strip(' '),
                            'remixer_url': remixer_url.strip(' ')
                            }
                track_docs[(track, artist, remixer)] = track_doc
                #print('\n\n\n')

    return track_docs

def build_artist_edges(url_doc, url):
    '''
    Build artist set-adjacency docs -- order n^2.
    Dont iterate over full set twice since will be considered non-directional
    '''
    all_tracks = []
    these_tracks = list(url_doc['track_docs'].values())
    for i in range(len(these_tracks)):
        for j in range(i,len(these_tracks)):

            track = these_tracks[i]
            other_track = these_tracks[j]

            first_artist = track['artist']
            second_artist = other_track['artist']

            if first_artist != second_artist:
                all_tracks.append(\
                                {
                                'artist1': first_artist,
                                'artist2': second_artist,
                                'url': url
                                }
                            )
    return all_tracks

def build_track_edges(track_docs, url):
    '''
    Build track set-adjacency docs -- order n^2.
    Dont iterate over full set twice since will be considered non-directional
    '''
    edge_docs = []
    keys = sorted(list(track_docs.keys()))
    for i in range(len(keys)):
        for j in range(i, len(keys)):
            
            key = keys[i]
            other_key = keys[j]
            
            if key != other_key:
                edge_docs.append(\
                                {
                                'track1': key,
                                'track2': other_key,
                                'url': url
                                }
                            )
    return edge_docs
                
def build_sequential_track_edges(track_docs, url):
    '''
    Allows for later "next track lookup" functionality
    
    '''
    enumerated_tracks = [(track_docs[key]['track_num'], key) for key in list(track_docs.keys())]
    enumerated_tracks = sorted(enumerated_tracks, key=lambda x: x[0])
    
    seq_docs = []
    for track_idx in range(len(enumerated_tracks)-1):
        seq_docs.append(\
                       {
                       'url': url,
                       'first_track': enumerated_tracks[track_idx][1],
                       'second_track': enumerated_tracks[track_idx+1][1],
                       'first_position': enumerated_tracks[track_idx][0],
                       'second_position': enumerated_tracks[track_idx+1][0],   
                       }
                    )
    return seq_docs

def build_played_playedby_edge(url_doc, url):
    '''
    Allows you to map who plays who.
    I think it would be interesting to study directional graphs from this.
    
    '''
    dj_name = url_doc['dj_name']
    dj_url = url_doc['dj_url']
    
    if (dj_name == 'N/A') or (dj_url == 'N/A'):
        return []
    
    played_docs = []
    for track_doc in list(url_doc['track_docs'].values()):
        
        played_docs.append(\
                          {
                          'url': url,
                          'played_by': dj_name,
                          'played_by_url': dj_url,
                          'played': track_doc['name'],
                          'played_track_url': track_doc['track_url'],
                          'played_artist': track_doc['artist'],
                          'played_artist_url': track_doc['artist_url'],
                          'played_remixer': track_doc['remixer'],
                          'played_remixer_url': track_doc['remixer_url']
                          }
                        )
    return played_docs
       

### Spot check parser

In [43]:
from urllib.request import Request, urlopen

url = 'https://www.1001tracklists.com/tracklist/2ujzkjmt/valentino-khan-cosmicmeadow-edc-las-vegas-united-states-2019-05-18.html'
req = Request(url,\
              headers={'User-Agent': 'Mozilla/5.0'})
html = str(urlopen(req).read())

url_doc = {}
url_doc['html'] = html
url_doc.update(tracklist_meta_data(html))
url_doc.update(tracklist_general_information(html))
track_docs = tracklist_track_data(html)
url_doc['track_docs'] = track_docs

track_edges = build_track_edges(track_docs, url)
print('Length of track connections:', len(track_edges))
sequential_edges = build_sequential_track_edges(track_docs, url)
print('Length of Sequential connections:', len(sequential_edges))
played_edges = build_played_playedby_edge(url_doc, url)
print('Length of played connections:', len(played_edges))
artist_edges = build_artist_edges(url_doc, url)
print('Length of artist connections:', len(artist_edges))

# url = 'https://www.1001tracklists.com/tracklist/21d7hr4t/lane-8-the-anjunadeep-edition-248-2019-04-18.html'
# req = Request(url,\
#               headers={'User-Agent': 'Mozilla/5.0'})
# html = str(urlopen(req).read())

# url_doc = {}
# url_doc['html'] = html
# url_doc.update(tracklist_meta_data(html))
# url_doc.update(tracklist_general_information(html))

# track_docs = tracklist_track_data(html)
# url_doc['track_docs'] = track_docs

# track_edges = build_track_edges(track_docs, url)
# print('Length of track connections:', len(track_edges))
# sequential_edges = build_sequential_track_edges(track_docs, url)
# print('Length of Sequential connections:', len(sequential_edges))
# played_edges = build_played_playedby_edge(url_doc, url)
# print('Length of played connections:', len(played_edges))
# artist_edges = build_artist_edges(url_doc, url)
# print('Length of artist connections:', len(artist_edges))

# url = 'https://www.1001tracklists.com/tracklist/2pvwxgkk/anjunadeep-the-anjunadeep-edition-with-tsha-249-2019-04-25.html'
# req = Request(url,\
#               headers={'User-Agent': 'Mozilla/5.0'})
# html = str(urlopen(req).read())

# url_doc = {}
# url_doc['html'] = html
# url_doc.update(tracklist_meta_data(html))
# url_doc.update(tracklist_general_information(html))

# track_docs = tracklist_track_data(html)
# url_doc['track_docs'] = track_docs

# track_edges = build_track_edges(track_docs, url)
# print('Length of track connections:', len(track_edges))
# sequential_edges = build_sequential_track_edges(track_docs)
# print('Length of Sequential connections:', len(sequential_edges))
# played_edges = build_played_playedby_edge(url_doc)
# print('Length of played connections:', len(played_edges))

# url = 'https://www.1001tracklists.com/tracklist/1nc1cquk/baile-the-anjunadeep-edition-250-2019-05-02.html'
# req = Request(url,\
#               headers={'User-Agent': 'Mozilla/5.0'})
# html = str(urlopen(req).read())

# url_doc = {}
# url_doc['html'] = html
# url_doc.update(tracklist_meta_data(html))
# url_doc.update(tracklist_general_information(html))

# track_docs = tracklist_track_data(html)
# url_doc['track_docs'] = track_docs

# track_edges = build_track_edges(track_docs, url)
# print('Length of track connections:', len(track_edges))
# sequential_edges = build_sequential_track_edges(track_docs)
# print('Length of Sequential connections:', len(sequential_edges))
# played_edges = build_played_playedby_edge(url_doc)
# print('Length of played connections:', len(played_edges))

# url = 'https://www.1001tracklists.com/tracklist/1kknyy01/qrion-the-anjunadeep-edition-251-2019-05-09.html'
# req = Request(url,\
#               headers={'User-Agent': 'Mozilla/5.0'})
# html = str(urlopen(req).read())    

# url_doc = {}
# url_doc['html'] = html
# url_doc.update(tracklist_meta_data(html))
# url_doc.update(tracklist_general_information(html))

# track_docs = tracklist_track_data(html)
# url_doc['track_docs'] = track_docs

# track_edges = build_track_edges(track_docs, url)
# print('Length of track connections:', len(track_edges))
# sequential_edges = build_sequential_track_edges(track_docs)
# print('Length of Sequential connections:', len(sequential_edges))
# played_edges = build_played_playedby_edge(url_doc)
# print('Length of played connections:', len(played_edges))

# url = 'https://www.1001tracklists.com/tracklist/pmj587k/james-grant-the-anjunadeep-edition-247-2019-04-11.html'
# req = Request(url,\
#               headers={'User-Agent': 'Mozilla/5.0'})
# html = str(urlopen(req).read())    

# url_doc = {}
# url_doc['html'] = html
# url_doc.update(tracklist_meta_data(html))
# url_doc.update(tracklist_general_information(html))

# track_docs = tracklist_track_data(html)
# url_doc['track_docs'] = track_docs

# track_edges = build_track_edges(track_docs, url)
# print('Length of track connections:', len(track_edges))
# sequential_edges = build_sequential_track_edges(track_docs)
# print('Length of Sequential connections:', len(sequential_edges))
# played_edges = build_played_playedby_edge(url_doc)
# print('Length of played connections:', len(played_edges))    

Length of track connections: 3081
Length of Sequential connections: 78
Length of played connections: 79
Length of artist connections: 3005


### Run over these and keep a depth of like 2 for POC

In [ ]:
# import time

# def find_str(s, char, start_index=0):

#     index = 0
#     s = s[start_index+1:]
#     if char in s:
#         c = char[0]
#         for ch in s:
#             if ch == c:
#                 if s[index:index+len(char)] == char:
#                     return start_index + 1 + index
#             index += 1
#     return -1   

# def request(url):

#     user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
#     req = Request(url,\
#                   headers={'User-Agent': user_agent})
#     html = str(urlopen(req).read())
#     return html

# date_pages = ["https://www.1001tracklists.com/index%d.html?order=date" %d for d in range(100)]
# urls = []
# for url in date_pages:
    
#     # Make http request
#     html = request(url)
#     index = 0
#     # Iterate over links found in html
#     count = 0
#     while find_str(html, 'a href="', index) != -1:
        
#         # Extract url
#         index = find_str(html, 'a href="', index)
#         url_chunk = html[index:].split('"')[1]

#         # Make sure it is either a referenced tracklist or 1001 page
#         if ('/tracklist/' in url_chunk) and ('http' not in url_chunk):
#             new_page = 'https://www.1001tracklists.com' + url_chunk
#             urls.append(new_page)
#         if ('www.1001tracklists.com' in url_chunk) and ('.html' in url_chunk):
#             urls.append(url_chunk)
            
#         count += 1
            
#     print('Number of URLS:', len(urls))
#     time.sleep(10)

# import pickle
# with open('1001_urls.pkl', 'wb') as f:
#      pickle.dump(urls, f)

In [11]:
# import numpy as np 
# import pickle

# with open('1001_urls.pkl', 'rb') as f:
#      urls = pickle.load(f)
        
# print(len(urls))
# unique_urls = np.unique(urls)
# print(len(unique_urls))
# unique_urls = [url for url in unique_urls if ('.html' in url) and ('#tlp' not in url)]
# print(len(unique_urls))

4534
2980
2970


In [ ]:
import time
import pickle
import numpy as np

def parse(url):
    
    req = Request(url,\
                  headers={'User-Agent': 'Mozilla/5.0'})
    html = str(urlopen(req).read())   
    
    url_doc = {}
    url_doc['url'] = url
    url_doc['html'] = html
    url_doc.update(tracklist_meta_data(html))
    url_doc.update(tracklist_general_information(html))

    track_docs = tracklist_track_data(html)
    url_doc['track_docs'] = track_docs

    track_edges = build_track_edges(track_docs, url)
    print('Length of track connections:', len(track_edges))
    sequential_edges = build_sequential_track_edges(track_docs, url)
    print('Length of Sequential connections:', len(sequential_edges))
    played_edges = build_played_playedby_edge(url_doc, url)
    print('Length of played connections:', len(played_edges))
    artist_edges = build_artist_edges(url_doc, url)
    print('Length of artist connections:', len(artist_edges))

    return url_doc, track_edges, sequential_edges, played_edges, artist_edges, html

    
played_docs = []
sequential_docs = []
track_docs = []
tracklist_docs = []
artist_docs = []
url_html_map = {}

with open('1001_urls.pkl', 'rb') as f:
    urls = pickle.load(f)

# Find unique urls
seen_urls = []
unique_urls = np.unique(urls)
unique_urls = [url for url in unique_urls\
                   if ('.html' in url) and ('#tlp' not in url) and (url not in seen_urls)]

for url in unique_urls[:]:
    
    if ('.html' in url) and ('#tlp' not in url) and (url not in seen_urls):
    
        try:
            
            url_doc, track_edges, sequential_edges, played_edges, artist_edges, html = parse(url)

            url_html_map[url] = html
            
            played_docs.extend(played_edges)
            sequential_docs.extend(sequential_edges)
            track_docs.extend(track_edges)
            tracklist_docs.append(url_doc)
            artist_docs.extend(artist_edges)

            print('Len played docs:', len(played_docs))
            print('Len sequential docs:', len(sequential_docs))
            print('Len track docs:', len(track_docs))
            print('Len tracklist docs:', len(tracklist_docs))
            print('Len artist docs:', len(artist_docs))
            
            with open('played_docs3.pkl', 'wb') as f:
                pickle.dump(played_docs, f)
            with open('sequential_docs3.pkl', 'wb') as f:
                pickle.dump(sequential_docs, f)
            with open('track_docs3.pkl', 'wb') as f:
                pickle.dump(track_docs, f)
            with open('tracklist_docs3.pkl', 'wb') as f:
                pickle.dump(tracklist_docs, f)
            with open('artist_docs3.pkl', 'wb') as f:
                pickle.dump(artist_docs, f)
            with open('url_html_map.pkl', 'wb') as f:
                pickle.dump(artist_docs, f)
                
            seen_urls.append(url)
            
        except Exception as e:
            print(e)
        
        print('waiting')
        time.sleep(10)

Length of track connections: 595
Length of Sequential connections: 34
Length of played connections: 35
Length of artist connections: 573
Len played docs: 35
Len sequential docs: 34
Len track docs: 595
Len tracklist docs: 1
Len artist docs: 573
waiting
Length of track connections: 105
Length of Sequential connections: 14
Length of played connections: 15
Length of artist connections: 101
Len played docs: 50
Len sequential docs: 48
Len track docs: 700
Len tracklist docs: 2
Len artist docs: 674
waiting
Length of track connections: 1953
Length of Sequential connections: 62
Length of played connections: 63
Length of artist connections: 1940
Len played docs: 113
Len sequential docs: 110
Len track docs: 2653
Len tracklist docs: 3
Len artist docs: 2614
waiting
Length of track connections: 153
Length of Sequential connections: 17
Length of played connections: 18
Length of artist connections: 140
Len played docs: 131
Len sequential docs: 127
Len track docs: 2806
Len tracklist docs: 4
Len artist d

Length of track connections: 105
Length of Sequential connections: 14
Length of played connections: 15
Length of artist connections: 102
Len played docs: 947
Len sequential docs: 937
Len track docs: 20328
Len tracklist docs: 33
Len artist docs: 19997
waiting
Length of track connections: 45
Length of Sequential connections: 9
Length of played connections: 10
Length of artist connections: 45
Len played docs: 957
Len sequential docs: 946
Len track docs: 20373
Len tracklist docs: 34
Len artist docs: 20042
waiting
Length of track connections: 780
Length of Sequential connections: 39
Length of played connections: 40
Length of artist connections: 779
Len played docs: 997
Len sequential docs: 985
Len track docs: 21153
Len tracklist docs: 35
Len artist docs: 20821
waiting
Length of track connections: 703
Length of Sequential connections: 37
Length of played connections: 38
Length of artist connections: 700
Len played docs: 1035
Len sequential docs: 1022
Len track docs: 21856
Len tracklist docs:

Length of track connections: 465
Length of Sequential connections: 30
Length of played connections: 31
Length of artist connections: 440
Len played docs: 1689
Len sequential docs: 1647
Len track docs: 30393
Len tracklist docs: 65
Len artist docs: 29878
waiting
Length of track connections: 253
Length of Sequential connections: 22
Length of played connections: 23
Length of artist connections: 253
Len played docs: 1712
Len sequential docs: 1669
Len track docs: 30646
Len tracklist docs: 66
Len artist docs: 30131
waiting
Length of track connections: 190
Length of Sequential connections: 19
Length of played connections: 20
Length of artist connections: 189
Len played docs: 1732
Len sequential docs: 1688
Len track docs: 30836
Len tracklist docs: 67
Len artist docs: 30320
waiting
Length of track connections: 406
Length of Sequential connections: 28
Length of played connections: 29
Length of artist connections: 405
Len played docs: 1761
Len sequential docs: 1716
Len track docs: 31242
Len trackl

Length of track connections: 435
Length of Sequential connections: 29
Length of played connections: 0
Length of artist connections: 431
Len played docs: 2391
Len sequential docs: 2371
Len track docs: 40588
Len tracklist docs: 97
Len artist docs: 39731
waiting
Length of track connections: 66
Length of Sequential connections: 11
Length of played connections: 12
Length of artist connections: 65
Len played docs: 2403
Len sequential docs: 2382
Len track docs: 40654
Len tracklist docs: 98
Len artist docs: 39796
waiting
Length of track connections: 276
Length of Sequential connections: 23
Length of played connections: 24
Length of artist connections: 270
Len played docs: 2427
Len sequential docs: 2405
Len track docs: 40930
Len tracklist docs: 99
Len artist docs: 40066
waiting
Length of track connections: 378
Length of Sequential connections: 27
Length of played connections: 28
Length of artist connections: 377
Len played docs: 2455
Len sequential docs: 2432
Len track docs: 41308
Len tracklist

Length of track connections: 210
Length of Sequential connections: 20
Length of played connections: 21
Length of artist connections: 210
Len played docs: 3150
Len sequential docs: 3111
Len track docs: 55082
Len tracklist docs: 129
Len artist docs: 54098
waiting
Length of track connections: 136
Length of Sequential connections: 16
Length of played connections: 17
Length of artist connections: 134
Len played docs: 3167
Len sequential docs: 3127
Len track docs: 55218
Len tracklist docs: 130
Len artist docs: 54232
waiting
Length of track connections: 78
Length of Sequential connections: 12
Length of played connections: 13
Length of artist connections: 78
Len played docs: 3180
Len sequential docs: 3139
Len track docs: 55296
Len tracklist docs: 131
Len artist docs: 54310
waiting
Length of track connections: 903
Length of Sequential connections: 42
Length of played connections: 43
Length of artist connections: 903
Len played docs: 3223
Len sequential docs: 3181
Len track docs: 56199
Len track

...

waiting
Length of track connections: 78
Length of Sequential connections: 12
Length of played connections: 0
Length of artist connections: 76
Len played docs: 16311
Len sequential docs: 16428
Len track docs: 276256
Len tracklist docs: 695
Len artist docs: 269433
waiting
Length of track connections: 45
Length of Sequential connections: 9
Length of played connections: 10
Length of artist connections: 45
Len played docs: 16321
Len sequential docs: 16437
Len track docs: 276301
Len tracklist docs: 696
Len artist docs: 269478
waiting
Length of track connections: 91
Length of Sequential connections: 13
Length of played connections: 14
Length of artist connections: 91
Len played docs: 16335
Len sequential docs: 16450
Len track docs: 276392
Len tracklist docs: 697
Len artist docs: 269569
waiting
Length of track connections: 190
Length of Sequential connections: 19
Length of played connections: 20
Length of artist connections: 190
Len played docs: 16355
Len sequential docs: 16469
Len track docs:

waiting
Length of track connections: 153
Length of Sequential connections: 17
Length of played connections: 18
Length of artist connections: 151
Len played docs: 17159
Len sequential docs: 17245
Len track docs: 292254
Len tracklist docs: 726
Len artist docs: 285210
waiting
Length of track connections: 465
Length of Sequential connections: 30
Length of played connections: 31
Length of artist connections: 464
Len played docs: 17190
Len sequential docs: 17275
Len track docs: 292719
Len tracklist docs: 727
Len artist docs: 285674
waiting
Length of track connections: 136
Length of Sequential connections: 16
Length of played connections: 17
Length of artist connections: 136
Len played docs: 17207
Len sequential docs: 17291
Len track docs: 292855
Len tracklist docs: 728
Len artist docs: 285810
waiting
Length of track connections: 630
Length of Sequential connections: 35
Length of played connections: 36
Length of artist connections: 554
Len played docs: 17243
Len sequential docs: 17326
Len tra

waiting
Length of track connections: 703
Length of Sequential connections: 37
Length of played connections: 38
Length of artist connections: 680
Len played docs: 17980
Len sequential docs: 18053
Len track docs: 305277
Len tracklist docs: 757
Len artist docs: 297912
waiting
Length of track connections: 153
Length of Sequential connections: 17
Length of played connections: 18
Length of artist connections: 142
Len played docs: 17998
Len sequential docs: 18070
Len track docs: 305430
Len tracklist docs: 758
Len artist docs: 298054
waiting
Length of track connections: 378
Length of Sequential connections: 27
Length of played connections: 28
Length of artist connections: 372
Len played docs: 18026
Len sequential docs: 18097
Len track docs: 305808
Len tracklist docs: 759
Len artist docs: 298426
waiting
Length of track connections: 231
Length of Sequential connections: 21
Length of played connections: 22
Length of artist connections: 231
Len played docs: 18048
Len sequential docs: 18118
Len tra

waiting
Length of track connections: 210
Length of Sequential connections: 20
Length of played connections: 21
Length of artist connections: 210
Len played docs: 18668
Len sequential docs: 18812
Len track docs: 319070
Len tracklist docs: 788
Len artist docs: 311554
waiting
Length of track connections: 45
Length of Sequential connections: 9
Length of played connections: 10
Length of artist connections: 45
Len played docs: 18678
Len sequential docs: 18821
Len track docs: 319115
Len tracklist docs: 789
Len artist docs: 311599
waiting
Length of track connections: 276
Length of Sequential connections: 23
Length of played connections: 24
Length of artist connections: 198
Len played docs: 18702
Len sequential docs: 18844
Len track docs: 319391
Len tracklist docs: 790
Len artist docs: 311797
waiting
Length of track connections: 210
Length of Sequential connections: 20
Length of played connections: 21
Length of artist connections: 210
Len played docs: 18723
Len sequential docs: 18864
Len track 

waiting
Length of track connections: 253
Length of Sequential connections: 22
Length of played connections: 0
Length of artist connections: 253
Len played docs: 19447
Len sequential docs: 19583
Len track docs: 333971
Len tracklist docs: 819
Len artist docs: 325638
waiting
Length of track connections: 66
Length of Sequential connections: 11
Length of played connections: 12
Length of artist connections: 60
Len played docs: 19459
Len sequential docs: 19594
Len track docs: 334037
Len tracklist docs: 820
Len artist docs: 325698
waiting
Length of track connections: 120
Length of Sequential connections: 15
Length of played connections: 16
Length of artist connections: 119
Len played docs: 19475
Len sequential docs: 19609
Len track docs: 334157
Len tracklist docs: 821
Len artist docs: 325817
waiting
Length of track connections: 66
Length of Sequential connections: 11
Length of played connections: 12
Length of artist connections: 66
Len played docs: 19487
Len sequential docs: 19620
Len track do

In [ ]:
all_df = pd.DataFrame(all_tracks)
all_df